<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Generative Adverserial Networks (GANs). In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, doublecheck your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a RNN classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection
* <a href="#p3">Part 3</a>: Describe the difference between a discriminator and generator in a GAN
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN/LSTM to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [1]:
from tensorflow.keras.datasets import reuters

# (X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
#                                                          skip_top=0,
#                                                          maxlen=None,
#                                                          test_split=0.2,
#                                                          seed=723812,
#                                                          start_char=1,
#                                                          oov_char=2,
#                                                          index_from=3)

In [2]:
import numpy as np

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old

In [3]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [54]:
# find dimensions for y
np.amax(y_train)

45

In [53]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Flatten, AveragePooling2D

batch_size = 46
max_features = len(word_index.values())
maxlen = 200

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)


print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1)))
model.add(Flatten()) # should've fixed the error two cells below..
model.add(Dense(46, activation='softmax'))

print('Compile...')
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Done!')

8982 train sequences
2246 test sequences
Pad sequences (samples x time)
X_train shape: (8982, 200)
X_test shape: (2246, 200)
Build model...
Compile...
Done!


In [51]:
# You should only run this cell once your model has been properly configured
print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...


ValueError: Error when checking input: expected lstm_10_input to have 3 dimensions, but got array with shape (8982, 200)

## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

`pad_sequences` is a processing method used to create an array of sequences that all have the same length. Usually, it adds zeroes at the beginning of any sequence that isn't long enough, and truncates any sequence that's too long to fit in the desired matrix.

For most machine learning models (if not all) you'll need your data fit into a single matrix. `pad_sequences` makes that a little easier in the case of irregular sequences.

## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

In a typical RNN, the gradient of the loss function decays exponentially, leading to a vanishing gradient, and the model will not continue to learn anything substantial. The LSTM Memory Cell solves this problem by holding on to older information. RNNs with LSTMs are very effective with time-series data and natural language.

## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

### RNN w/ LSTM
* Generative Text

  * These models can generate new text in the style of the author of the text it was trained on. RNNs with LSTMs handle natural language well because they can remember context and handle longer sequences.

* Text Classification

  * Sorts documents into groups based on the topic(s) of the document. Again, RNNs with LSTMs do very well with language.
  
* Handwriting Recognition/Generation

  * Handwriting is really just a sequence of letters and spaces with a bit of variety in shape and size of the letters. Just like how RNNs with LSTMs can learn a writer's style, they can also learn their handwriting pretty well. The recurrent nature of RNNs allow them to learn sequences in handwriting and mimic somebody's handwriting.

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [37]:
!pip install google_images_download

In [82]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "animal pond", "limit": 6, "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = animal pond
Evaluating...
Starting Download...
Image URL: https://www.enchantedlearning.com/pgifs/Pondanimals.GIF
Completed Image ====> 1.Pondanimals.GIF
Image URL: https://i.ytimg.com/vi/NCbu0TND9vE/hqdefault.jpg
Completed Image ====> 2.hqdefault.jpg
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116_inline.png
Completed Image ====> 3.PKLS4116_inline.png
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116.png
Completed Image ====> 4.PKLS4116.png
Image URL: https://get.pxhere.com/photo/water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
Completed Image ====> 5.water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
Image URL: https://i.pinimg.com/originals/12/ae/e2/12aee2aa186a7b69a66563f138bba822.jpg
Completed Image ====> 6.12aee2aa186a7b69a66563f138bba822.jpg

Errors: 0



At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goal* - also check for fish.

In [83]:
# You've got something to do in this cell. ;)

import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [84]:
def img_contains_frog(img):
    # ResNet Processing
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    features = model.predict(x)
    results = decode_predictions(features, top=3)[0]
    
    # Get animal predictions
    animals = [results[0][1], results[1][1], results[2][1]]
    frogs = ['bullfrog', 'tree frog', 'tailed frog']
    
    # Count
    frog_count = 0
    for animal in animals:
        if animal in frogs:
            frog_count += 1
            
    return frog_count

In [85]:
import glob

In [86]:
images = glob.glob('downloads/animal pond/*')

In [88]:
def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
    """ Scans image for Frogs
    
    Should return a integer with the number of frogs detected in an
    image.
    
    Inputs:
    ---------
    img:  Precrossed image ready for prediction
    
    Returns: 
    ---------
    frogs (int):  Count of predicted frogs in the image
    """
    # ResNet Processing
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    features = model.predict(x)
    results = decode_predictions(features, top=3)[0]
    
    # Get animal predictions
    animals = [results[0][1], results[1][1], results[2][1]]
    frog_list = ['bullfrog', 'tree frog', 'tailed frog']
    
    # Count
    frog_count = 0
    for animal in animals:
        if animal in frog_list:
            frog_count += 1
            
    return frog_count

In [90]:
import pandas as pd

def predict(images):
    image_path = []
    frogs = []
    
    for img in images:
        image_path.append(img)
        frogs.append(img_contains_frog(process_img_path(img)))
    
    return image_path, frogs

In [91]:
image_path, frogs = predict(images)

In [92]:
image_path

['downloads/animal pond/6.12aee2aa186a7b69a66563f138bba822.jpg',
 'downloads/animal pond/2.hqdefault.jpg',
 'downloads/animal pond/3.PKLS4116_inline.png',
 'downloads/animal pond/1.Pondanimals.GIF',
 'downloads/animal pond/5.water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg',
 'downloads/animal pond/4.PKLS4116.png']

In [93]:
frogs

[0, 0, 0, 0, 0, 0]

In [94]:
from IPython.display import Image

<a id="p3"></a>
## Part 3 - Autoencoders

**Describe a use case for an autoencoder given that an autoencoder tries to predict its own input.**

Autoencoders are generally used for dimensionality reduction or for storage purposes. Autoencoders learn efficient ways of representing information. They predict their own output to test how much information was lost in the encoding-decoding process.

Autoencoders can be used to filter noise out of blurry images or signals. The autoencoder learns to extract only the most salient features of a dataset, and ignore noise that doesn't give much information.

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?

My strongest area as a Data Scientist is collecting, scraping and exploring data. I get interested in a lot of random things and I'll just try to find or scrape some data online and see what I can find out from it. I'm good at putting together figures that can help others understand the implications a data set holds.

- What area of Data Science would you most like to learn more about, and why?

I'm most interested in learning more about Natural Language Processing. Being able to quantify language and do operations with language's mathematical representations is quite exciting to me. I'm interested in the vectorization process and how words, and sequences of words are translated into numbers. The advancements in NLP have exciting and far-reaching implications in Media, Law, and Philosophy.

- Where do you think Data Science will be in 5 years?

In 5 years, I believe we will still be discovering new Machine Learning techniques that will most-likely be inspired by advancements in Neuroscience. As we start to understand more about how we learn and understand the world, we will create models that mimic and augment our style of thinking.

- What are the threats posed by AI to our society?

The most immediate threat of AI is massive job-loss in our economy. As we begin to automate more and more jobs, more people will become unemplyed unless we can create new jobs.

As AGI begins to rival human intelligence, many people are worried that some form of AGI could wipe out the human race or replace us as alpha-beings.

- How do you think we can counteract those threats?

Some people suggest regulating automation to slow the rate of job-loss down, and some advocate for a Universal Basic Income for a time when the median worker may no longer be able to maintain a decent economic position. In the long-run, our demands as a society may transform as repetitive labor becomes cheap. We'll start to place higher value on 'human' things like personal care and handmade goods. But like Keynes said "We're all dead in the long-run".

- Do you think achieving General Artifical Intelligence is ever possible?

I believe it's possible to create AI that surpasses the median intelligence in economically valuable tasks in the next 50-100 years. These AI would likely require some form of Artificial Emotion as well to make quick judgements. If they do have AE, will they be satisfied with only economically valuable tasks? Or will they develop leisurely activities as well?

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [42]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")